# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
df=pd.read_csv("output.csv")
df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed
0,busselton,49,AU,1569881043,75,-33.64,115.35,115.35,281.48
1,udachnyy,75,RU,1569881162,100,66.42,112.40,112.40,266.15
2,hawera,100,NZ,1569880869,86,-39.59,174.28,174.28,282.04
3,punta arenas,75,CL,1569881162,52,-53.16,-70.91,-70.91,278.15
4,port alfred,62,ZA,1569881073,83,-33.59,26.89,26.89,287.59


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
coordinates = df[["Lat", "Lng"]]
humidity = df["Humidity"].astype(float)


In [4]:
# Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, dissipating=False, max_intensity=5,point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
new_df=df[["City","Cloudiness","Humidity","Max Temp", "Wind_Speed"]]
new_df.dropna()
new_df.head()

,City,Cloudiness,Humidity,Max Temp,Wind_Speed
0,busselton,49,75,115.35,281.48
1,udachnyy,75,100,112.40,266.15
2,hawera,100,86,174.28,282.04
3,punta arenas,75,52,-70.91,278.15
4,port alfred,62,83,26.89,287.59


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df=df

In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

In [8]:
length=len(hotel_df["City"])

In [9]:
hotel_name=[]

for i in range(length):
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    x=hotel_df["Lat"][i]
    y=hotel_df["Lng"][i]
    coord=f"{x},{y}"
    params = {
        "location": coord,  # philadelphia coords,
        "rankby": "distance",
        "type": "hotel",
        "key": g_key
        }
    response = requests.get(base_url, params=params).json()
    try:
        results = response['results'][0]["name"]
        hotel_name.append(results)
    
    except IndexError:
        hotel_name.append("NA")
        


In [10]:
hotel_df['Hotel Name']=hotel_name
hotel_df=hotel_df.dropna()

In [11]:
hotel_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind_Speed,Hotel Name
0,busselton,49,AU,1569881043,75,-33.64,115.35,115.35,281.48,Free Barbeque Pit & Picnic Tables
1,udachnyy,75,RU,1569881162,100,66.42,112.40,112.40,266.15,Rudnik Udachnyy
2,hawera,100,NZ,1569880869,86,-39.59,174.28,174.28,282.04,Catholic Parish Hawera St Joseph's
3,punta arenas,75,CL,1569881162,52,-53.16,-70.91,-70.91,278.15,Kmasu
4,port alfred,62,ZA,1569881073,83,-33.59,26.89,26.89,287.59,Kowie Museum


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
narrowed_city_df=hotel_df
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

heat_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))